In [1]:
## This file implements neural network for p002ykpresabs_qual with four replicates.
## We compute the mean and standarad deviation of training and test accuracies.
## We also compute the mean and standard deviation of AUC ROC values for four replicates.

In [2]:
from numpy.random import seed
import numpy as np
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [3]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/p002ykpresabs_qual.csv')
df.shape

(253, 1736)

In [4]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [5]:
df['pheno']

0      0
1      0
2      1
3      0
4      0
      ..
248    0
249    0
250    0
251    0
252    0
Name: pheno, Length: 253, dtype: int64

In [6]:
df.head()

,id,TTTTTTTGACTAAAATTAATGAAAAGTGAAAATAGTATTGGAACTCAATATCTTTAATGATTTAATGAATAATTTTTATTGAAAGCGATAATTCGTATTA,TTTTTTTATGAAT,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATG,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGA,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGATGTAAAGCGTAA,TTTTTTGACTAAAATTAATGAAAAGTGAAAATAGTATTGGAACTCAATATCTTTAATGATTTAATGAATAATTTTTATTGAAAGCGATAATTCGTATTAA,TTTTTTCTTTTCATAACTGTGTTGGAAATGAATTAAATTAACAGCTCTTTGTGCTTTACGGTGTGTTGC,TTTTTTCATTAGT,TTTTTTCATTAGTAA,...,AAGCATGTCTT,AACTAGGGGGGATTAGAATGCAAAATAAGTCAAAATCGCCTTTTAAAATTGCATTTTCTAAATTTATTCATAATAAAATTGCAATGTTATCGGTTATTTT,AACCTTAAAATTAACTTCTTTCAATTTCATAATAAAGTCTCTATAAAATAACTTAGTTTAAAAACGATTCGTATCTTTCAGATTCAAATACCATCATTTT,AACAAACTCAATTAATACGAATTATCGCTTTCAATAAAAATTATTCATTAAATCATTAAAGATATTGAGTTCCAATACTATTTTCACTTTTCATTAATTT,AAATTTTTCAACAAACTCAATTAATACGAATTATCGCTTTCAATAAAAATTATTCATTAAATCATTAAAGATATTGAGTTCCAATACTATTTTCACTTTT,AAATTTTACTTAACAACTAGTACATAAAGTAATACAATTAAATTAATTCTATCTGAAAGATGTGTGGGGCATCGTTATTTTAGGTGGATATGAGCAATTT,AAATTGCGTATTT,AAATGCAGTAAAATTAATCGAGAGGTGCCATTTTGAAGACAGGTCGAATAGTGAAATCAATTAGTGGGGTATATCAAGTAGACGTTAATGGCGAACGTTT,AAATCTTAGACGTAAACATGATAAAATGGCCTTGATTACTCAATAGTTATATTTCGGAGAACTGATTTGTGATATGATATTAAAGACTATAGGAGGATTT,pheno
0,107,0,1,1,1,1,0,0,1,1,...,1,0,0,0,0,0,0,1,0,0
1,109,0,1,1,1,1,0,0,1,1,...,1,0,0,0,0,0,0,1,0,0
2,115,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3,120335,0,1,1,1,1,0,0,1,1,...,1,0,0,0,0,0,0,1,0,0
4,120337,0,1,1,1,1,0,0,1,1,...,1,0,0,0,0,0,0,1,0,0


In [7]:
df['pheno'].value_counts()

0    220
1     30
2      3
Name: pheno, dtype: int64

In [8]:
df_clean = df.drop(columns=['id'])

In [9]:
df_clean.shape

(253, 1735)

In [10]:
df_clean.head()

,TTTTTTTGACTAAAATTAATGAAAAGTGAAAATAGTATTGGAACTCAATATCTTTAATGATTTAATGAATAATTTTTATTGAAAGCGATAATTCGTATTA,TTTTTTTATGAAT,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATG,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGA,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGATGTAAAGCGTAA,TTTTTTGACTAAAATTAATGAAAAGTGAAAATAGTATTGGAACTCAATATCTTTAATGATTTAATGAATAATTTTTATTGAAAGCGATAATTCGTATTAA,TTTTTTCTTTTCATAACTGTGTTGGAAATGAATTAAATTAACAGCTCTTTGTGCTTTACGGTGTGTTGC,TTTTTTCATTAGT,TTTTTTCATTAGTAA,TTTTTTCAGCATTGTCTACATTACTTAACATTCGTGTTTGTAAGTAATATTGACCGCCAATATTTAGACACTTTATAAGTATGCCATTCATCATTTTTAA,...,AAGCATGTCTT,AACTAGGGGGGATTAGAATGCAAAATAAGTCAAAATCGCCTTTTAAAATTGCATTTTCTAAATTTATTCATAATAAAATTGCAATGTTATCGGTTATTTT,AACCTTAAAATTAACTTCTTTCAATTTCATAATAAAGTCTCTATAAAATAACTTAGTTTAAAAACGATTCGTATCTTTCAGATTCAAATACCATCATTTT,AACAAACTCAATTAATACGAATTATCGCTTTCAATAAAAATTATTCATTAAATCATTAAAGATATTGAGTTCCAATACTATTTTCACTTTTCATTAATTT,AAATTTTTCAACAAACTCAATTAATACGAATTATCGCTTTCAATAAAAATTATTCATTAAATCATTAAAGATATTGAGTTCCAATACTATTTTCACTTTT,AAATTTTACTTAACAACTAGTACATAAAGTAATACAATTAAATTAATTCTATCTGAAAGATGTGTGGGGCATCGTTATTTTAGGTGGATATGAGCAATTT,AAATTGCGTATTT,AAATGCAGTAAAATTAATCGAGAGGTGCCATTTTGAAGACAGGTCGAATAGTGAAATCAATTAGTGGGGTATATCAAGTAGACGTTAATGGCGAACGTTT,AAATCTTAGACGTAAACATGATAAAATGGCCTTGATTACTCAATAGTTATATTTCGGAGAACTGATTTGTGATATGATATTAAAGACTATAGGAGGATTT,pheno
0,0,1,1,1,1,0,0,1,1,0,...,1,0,0,0,0,0,0,1,0,0
1,0,1,1,1,1,0,0,1,1,0,...,1,0,0,0,0,0,0,1,0,0
2,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3,0,1,1,1,1,0,0,1,1,0,...,1,0,0,0,0,0,0,1,0,0
4,0,1,1,1,1,0,0,1,1,0,...,1,0,0,0,0,0,0,1,0,0


In [11]:
X = df.loc[:, df.columns != 'pheno']
y = df['pheno']
print(X.shape, y.shape)

(253, 1735) (253,)


In [12]:
# over-sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.bagging module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.base module is  deprecated in versi

[(0, 220), (1, 220), (2, 220)]


/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is depreca

In [13]:
############# Fully-Connected Neural Network ################

In [14]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1

In [15]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [16]:
# pd.DataFrame(y_test_over).to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/ytest_p002.csv",
#                                header=None, index=None)

In [17]:
dat = pd.DataFrame(X_test_over[:,0])
dat['test'] = y_test_over

In [18]:
dat

,0,test
0,CFBRSa26,0
1,NRS109,2
2,NRS112,0
3,NRS216,1
4,NRS021,0
...,...,...
193,CFBREBSa133,0
194,NRS209,2
195,NRS109,2
196,NRS209,2


In [19]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [103]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [104]:
model1_over.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [105]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 462 samples, validate on 198 samples
Epoch 1/100
462/462 [==============================] - 0s 913us/step - loss: 0.9670 - accuracy: 0.5758 - val_loss: 0.6098 - val_accuracy: 0.6919
Epoch 2/100
462/462 [==============================] - 0s 202us/step - loss: 0.5866 - accuracy: 0.7403 - val_loss: 0.5312 - val_accuracy: 0.7172
Epoch 3/100
462/462 [==============================] - 0s 209us/step - loss: 0.5077 - accuracy: 0.7684 - val_loss: 0.4943 - val_accuracy: 0.7828
Epoch 4/100
462/462 [==============================] - 0s 184us/step - loss: 0.4838 - accuracy: 0.7879 - val_loss: 0.4751 - val_accuracy: 0.7980
Epoch 5/100
462/462 [==============================] - 0s 178us/step - loss: 0.4732 - accuracy: 0.8030 - val_loss: 0.4488 - val_accuracy: 0.7929
Epoch 6/100
462/462 [==============================] - 0s 175us/step - loss: 0.4599 - accuracy: 0.7749 - val_loss: 0.4578 - val_accuracy: 0.8030
Epoch 7/100
462/462 [==============================] - 0s 178us/step - loss: 0.4571 

Epoch 57/100
462/462 [==============================] - 0s 178us/step - loss: 0.2598 - accuracy: 0.9026 - val_loss: 0.2624 - val_accuracy: 0.8838
Epoch 58/100
462/462 [==============================] - 0s 157us/step - loss: 0.2610 - accuracy: 0.8701 - val_loss: 0.2741 - val_accuracy: 0.9091
Epoch 59/100
462/462 [==============================] - 0s 151us/step - loss: 0.2443 - accuracy: 0.9026 - val_loss: 0.3115 - val_accuracy: 0.8838
Epoch 60/100
462/462 [==============================] - 0s 155us/step - loss: 0.2944 - accuracy: 0.8636 - val_loss: 0.2968 - val_accuracy: 0.9091
Epoch 61/100
462/462 [==============================] - 0s 149us/step - loss: 0.2541 - accuracy: 0.8961 - val_loss: 0.2401 - val_accuracy: 0.9040
Epoch 62/100
462/462 [==============================] - 0s 151us/step - loss: 0.2541 - accuracy: 0.9048 - val_loss: 0.2483 - val_accuracy: 0.8889
Epoch 63/100
462/462 [==============================] - 0s 156us/step - loss: 0.2378 - accuracy: 0.9026 - val_loss: 0.2698 -

In [209]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

198/198 [==============================] - 0s 197us/step
over-sampling test accuracy: 90.91%


In [106]:
pred = model1_over.predict_classes(X_test_over)
pred

array([0, 2, 0, 1, 0, 0, 2, 2, 2, 0, 1, 0, 1, 1, 0, 1, 2, 1, 0, 1, 0, 0,
       1, 1, 1, 0, 1, 2, 2, 1, 0, 0, 2, 0, 2, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       2, 2, 2, 1, 2, 2, 2, 1, 0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 2, 0, 0, 1,
       0, 1, 0, 2, 0, 0, 1, 1, 2, 1, 0, 1, 2, 0, 1, 2, 0, 0, 2, 0, 1, 0,
       0, 2, 0, 0, 2, 0, 1, 2, 1, 1, 1, 2, 0, 2, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 0, 1, 0, 1, 0, 1, 1, 2, 2, 2, 2, 2, 2, 0, 2, 1, 0, 2, 0, 0,
       2, 2, 1, 2, 1, 2, 2, 2, 2, 1, 0, 2, 0, 0, 2, 0, 0, 2, 2, 0, 2, 1,
       0, 2, 0, 2, 1, 0, 2, 1, 1, 2, 0, 1, 2, 2, 1, 0, 2, 2, 2, 1, 2, 2,
       1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 2, 0, 2, 1, 2, 0, 2, 2, 2, 1])

In [107]:
dat['pred'] = pred
dat

,0,test,pred
0,CFBRSa26,0,0
1,NRS109,2,2
2,NRS112,0,0
3,NRS216,1,1
4,NRS021,0,0
...,...,...,...
193,CFBREBSa133,0,0
194,NRS209,2,2
195,NRS109,2,2
196,NRS209,2,2


In [108]:
proba1 = model1_over.predict_proba(X_test_over)
dat_proba1 = pd.DataFrame(proba1)

In [109]:
dat_proba1

,0,1,2
0,7.880825e-01,0.211917,5.676106e-07
1,7.316293e-03,0.029887,9.627965e-01
2,7.342295e-01,0.265767,3.070897e-06
3,1.440693e-01,0.855900,3.096955e-05
4,9.177732e-01,0.082227,1.213062e-09
...,...,...,...
193,9.688821e-01,0.010640,2.047772e-02
194,8.707785e-07,0.000050,9.999487e-01
195,7.316300e-03,0.029887,9.627965e-01
196,8.707785e-07,0.000050,9.999487e-01


In [27]:
dat_proba1.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba1.csv", index = False,
         header=None)

In [28]:
dat.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/1p002.csv", index = False,
         header=None)

In [213]:
hist1_over = model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 462 samples, validate on 198 samples
Epoch 1/100
462/462 [==============================] - 0s 272us/step - loss: 0.2194 - accuracy: 0.9069 - val_loss: 0.2123 - val_accuracy: 0.8990
Epoch 2/100
462/462 [==============================] - 0s 192us/step - loss: 0.2217 - accuracy: 0.9113 - val_loss: 0.2250 - val_accuracy: 0.9141
Epoch 3/100
462/462 [==============================] - 0s 185us/step - loss: 0.2251 - accuracy: 0.9113 - val_loss: 0.2228 - val_accuracy: 0.9040
Epoch 4/100
462/462 [==============================] - 0s 180us/step - loss: 0.2257 - accuracy: 0.9134 - val_loss: 0.2436 - val_accuracy: 0.8737
Epoch 5/100
462/462 [==============================] - 0s 191us/step - loss: 0.2400 - accuracy: 0.8961 - val_loss: 0.2321 - val_accuracy: 0.8939
Epoch 6/100
462/462 [==============================] - 0s 197us/step - loss: 0.2296 - accuracy: 0.9004 - val_loss: 0.2170 - val_accuracy: 0.8939
Epoch 7/100
462/462 [==============================] - 0s 176us/step - loss: 0.2368 

Epoch 57/100
462/462 [==============================] - 0s 159us/step - loss: 0.2190 - accuracy: 0.9134 - val_loss: 0.2119 - val_accuracy: 0.9040
Epoch 58/100
462/462 [==============================] - 0s 152us/step - loss: 0.2163 - accuracy: 0.9091 - val_loss: 0.2210 - val_accuracy: 0.9040
Epoch 59/100
462/462 [==============================] - 0s 174us/step - loss: 0.2184 - accuracy: 0.9069 - val_loss: 0.2152 - val_accuracy: 0.8990
Epoch 60/100
462/462 [==============================] - 0s 179us/step - loss: 0.2120 - accuracy: 0.9177 - val_loss: 0.2228 - val_accuracy: 0.8990
Epoch 61/100
462/462 [==============================] - 0s 183us/step - loss: 0.2214 - accuracy: 0.9091 - val_loss: 0.2083 - val_accuracy: 0.9040
Epoch 62/100
462/462 [==============================] - 0s 201us/step - loss: 0.2264 - accuracy: 0.9134 - val_loss: 0.2285 - val_accuracy: 0.9040
Epoch 63/100
462/462 [==============================] - 0s 225us/step - loss: 0.2312 - accuracy: 0.8896 - val_loss: 0.2304 -

In [214]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over.history['accuracy'])*100))

over-sampling train accuracy: 90.63%


In [20]:
df_proba = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=0,
                        index_col=None)

In [21]:
df_proba

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,CFBRSa26,0,0,0.758914,0.241086,4.638713e-07
1,p002ykpresabs_qual,NRS109,2,2,0.005361,0.016236,9.784034e-01
2,p002ykpresabs_qual,NRS112,0,0,0.726623,0.273376,1.520979e-06
3,p002ykpresabs_qual,NRS216,1,1,0.138322,0.861665,1.334123e-05
4,p002ykpresabs_qual,NRS021,0,0,0.882176,0.117824,1.414530e-10
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS148,2,2,0.000007,0.000099,9.998934e-01
4280,pyopresabsSTCC_qual,NRS255,2,2,0.000257,0.002048,9.976944e-01
4281,pyopresabsSTCC_qual,NRS205,2,2,0.000011,0.000045,9.999435e-01
4282,pyopresabsSTCC_qual,NRS255,2,2,0.000257,0.002048,9.976944e-01


In [22]:
y_prob = df_proba[df_proba['phage']=='p002ykpresabs_qual'].iloc[:,-3:]
y_prob = y_prob.to_numpy()
y_prob

array([[7.58913900e-01, 2.41085740e-01, 4.63871320e-07],
       [5.36052320e-03, 1.62360100e-02, 9.78403450e-01],
       [7.26622900e-01, 2.73375570e-01, 1.52097860e-06],
       [1.38321760e-01, 8.61664950e-01, 1.33412270e-05],
       [8.82175800e-01, 1.17824230e-01, 1.41452960e-10],
       [7.58913900e-01, 2.41085740e-01, 4.63871320e-07],
       [1.46461300e-04, 3.05915720e-03, 9.96794400e-01],
       [1.46461300e-04, 3.05915720e-03, 9.96794400e-01],
       [5.36052320e-03, 1.62360100e-02, 9.78403450e-01],
       [9.99430840e-01, 5.69082000e-04, 1.12166800e-09],
       [2.11253200e-02, 9.78849500e-01, 2.51931940e-05],
       [8.82175800e-01, 1.17824230e-01, 1.41452960e-10],
       [1.24289480e-01, 8.75684400e-01, 2.61002660e-05],
       [2.32349190e-02, 9.44656400e-01, 3.21087050e-02],
       [8.82175800e-01, 1.17824230e-01, 1.41452960e-10],
       [1.03466480e-01, 8.96517300e-01, 1.62465200e-05],
       [1.46461300e-04, 3.05915720e-03, 9.96794400e-01],
       [1.77887400e-02, 9.82177

In [23]:
## Retrieved from https://github.com/scikit-learn/scikit-learn/issues/3298
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelBinarizer

def rocauc_ovo(truth, pred, average="macro", multi_class="ovo"):

    lb = LabelBinarizer()
    lb.fit(truth)

    truth = lb.transform(truth)   
    
    return roc_auc_score(truth, pred, average=average, multi_class=multi_class)

In [27]:
ovo1 = rocauc_ovo(y_test_over, y_prob, average="macro", multi_class="ovo")
ovo1

0.9682812978267523

In [25]:
def rocauc_ovr(truth, pred, average="macro", multi_class="ovr"):

    lb = LabelBinarizer()
    lb.fit(truth)

    truth = lb.transform(truth)   

    return roc_auc_score(truth, pred, average=average, multi_class=multi_class)

In [28]:
ovr1 = rocauc_ovr(y_test_over, y_prob, average="macro", multi_class="ovr")
ovr1

0.9682812978267523

In [32]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=125,
                                                    stratify=y_over)

In [33]:
dat2 = pd.DataFrame(X_test_over[:,0])
dat2['test'] = y_test_over

In [34]:
dat2

,0,test
0,NRS148,2
1,BCH-SA-03,1
2,NRS218,1
3,NRS036,0
4,NRS386,1
...,...,...
193,NRS169,1
194,NRS148,2
195,NRS198,0
196,CFBREBSa131,1


In [35]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [33]:
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [34]:
model1_over2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [35]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=64, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 462 samples, validate on 198 samples
Epoch 1/100
462/462 [==============================] - 0s 344us/step - loss: 0.7322 - accuracy: 0.6450 - val_loss: 0.5270 - val_accuracy: 0.7626
Epoch 2/100
462/462 [==============================] - 0s 137us/step - loss: 0.5224 - accuracy: 0.7814 - val_loss: 0.4920 - val_accuracy: 0.7879
Epoch 3/100
462/462 [==============================] - 0s 130us/step - loss: 0.5127 - accuracy: 0.7662 - val_loss: 0.4676 - val_accuracy: 0.8081
Epoch 4/100
462/462 [==============================] - 0s 129us/step - loss: 0.4782 - accuracy: 0.7511 - val_loss: 0.4773 - val_accuracy: 0.7778
Epoch 5/100
462/462 [==============================] - 0s 141us/step - loss: 0.4536 - accuracy: 0.7662 - val_loss: 0.4405 - val_accuracy: 0.8030
Epoch 6/100
462/462 [==============================] - 0s 144us/step - loss: 0.4359 - accuracy: 0.8203 - val_loss: 0.4630 - val_accuracy: 0.7929
Epoch 7/100
462/462 [==============================] - 0s 163us/step - loss: 0.4366 

Epoch 57/100
462/462 [==============================] - 0s 217us/step - loss: 0.2794 - accuracy: 0.8896 - val_loss: 0.3003 - val_accuracy: 0.8636
Epoch 58/100
462/462 [==============================] - 0s 194us/step - loss: 0.2747 - accuracy: 0.8788 - val_loss: 0.3018 - val_accuracy: 0.8636
Epoch 59/100
462/462 [==============================] - 0s 141us/step - loss: 0.2700 - accuracy: 0.8658 - val_loss: 0.3092 - val_accuracy: 0.8889
Epoch 60/100
462/462 [==============================] - 0s 120us/step - loss: 0.2604 - accuracy: 0.8723 - val_loss: 0.2853 - val_accuracy: 0.8737
Epoch 61/100
462/462 [==============================] - 0s 132us/step - loss: 0.2525 - accuracy: 0.9113 - val_loss: 0.3073 - val_accuracy: 0.8283
Epoch 62/100
462/462 [==============================] - 0s 159us/step - loss: 0.2683 - accuracy: 0.8939 - val_loss: 0.2939 - val_accuracy: 0.8485
Epoch 63/100
462/462 [==============================] - 0s 388us/step - loss: 0.2502 - accuracy: 0.8853 - val_loss: 0.3402 -

In [282]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

198/198 [==============================] - 0s 169us/step
over-sampling test accuracy: 87.88%


In [36]:
pred2 = model1_over2.predict_classes(X_test_over)
pred2

array([2, 0, 1, 0, 0, 1, 1, 2, 1, 2, 2, 0, 1, 0, 1, 0, 2, 2, 1, 2, 0, 0,
       0, 0, 0, 2, 0, 0, 0, 2, 0, 2, 2, 0, 0, 1, 0, 0, 2, 0, 0, 0, 2, 1,
       0, 1, 2, 2, 1, 0, 0, 0, 0, 1, 2, 2, 0, 1, 2, 1, 1, 2, 0, 2, 1, 0,
       1, 0, 1, 2, 0, 2, 0, 0, 1, 0, 1, 0, 0, 2, 0, 2, 1, 2, 2, 0, 2, 2,
       2, 1, 2, 2, 0, 0, 0, 0, 1, 0, 2, 2, 1, 1, 2, 0, 1, 2, 1, 1, 2, 2,
       1, 2, 0, 2, 2, 0, 0, 1, 2, 1, 2, 0, 1, 1, 2, 2, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 2, 0, 2, 2, 2, 1, 0, 0, 1, 2, 0, 2, 1, 0, 2, 1, 0, 1,
       0, 2, 1, 0, 1, 2, 0, 0, 2, 2, 0, 1, 0, 2, 2, 1, 2, 1, 2, 1, 1, 0,
       0, 0, 2, 0, 2, 2, 1, 0, 1, 0, 2, 0, 0, 0, 2, 0, 0, 1, 2, 0, 0, 2])

In [37]:
dat2['pred'] = pred2
dat2

,0,test,pred
0,NRS148,2,2
1,BCH-SA-03,1,0
2,NRS218,1,1
3,NRS036,0,0
4,NRS386,1,0
...,...,...,...
193,NRS169,1,1
194,NRS148,2,2
195,NRS198,0,0
196,CFBREBSa131,1,0


In [38]:
proba2 = model1_over2.predict_proba(X_test_over)
dat_proba2 = pd.DataFrame(proba2)

In [39]:
dat_proba2

,0,1,2
0,0.000056,0.001748,9.981959e-01
1,0.712007,0.287992,9.646217e-07
2,0.006222,0.993773,4.482882e-06
3,0.882617,0.117383,2.310933e-10
4,0.571179,0.428818,2.444667e-06
...,...,...,...
193,0.029760,0.947224,2.301580e-02
194,0.000056,0.001748,9.981959e-01
195,0.968358,0.031642,1.288276e-10
196,0.968358,0.031642,1.288276e-10


In [40]:
dat_proba2.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba2.csv", index = False,
         header=None)

In [41]:
dat2.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/2p002.csv", index = False,
         header=None)

In [286]:
hist1_over2 = model1_over2.fit(X_train_over, y_train_over,
          batch_size=64, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 462 samples, validate on 198 samples
Epoch 1/100
462/462 [==============================] - 0s 194us/step - loss: 0.2317 - accuracy: 0.9048 - val_loss: 0.2651 - val_accuracy: 0.8889
Epoch 2/100
462/462 [==============================] - 0s 141us/step - loss: 0.2288 - accuracy: 0.9048 - val_loss: 0.2516 - val_accuracy: 0.8737
Epoch 3/100
462/462 [==============================] - 0s 127us/step - loss: 0.2165 - accuracy: 0.9113 - val_loss: 0.2572 - val_accuracy: 0.8788
Epoch 4/100
462/462 [==============================] - 0s 135us/step - loss: 0.2192 - accuracy: 0.9134 - val_loss: 0.2691 - val_accuracy: 0.8939
Epoch 5/100
462/462 [==============================] - 0s 134us/step - loss: 0.2352 - accuracy: 0.9091 - val_loss: 0.3092 - val_accuracy: 0.8434
Epoch 6/100
462/462 [==============================] - 0s 263us/step - loss: 0.2563 - accuracy: 0.8896 - val_loss: 0.2775 - val_accuracy: 0.8535
Epoch 7/100
462/462 [==============================] - 0s 224us/step - loss: 0.2248 

Epoch 57/100
462/462 [==============================] - 0s 181us/step - loss: 0.1926 - accuracy: 0.9134 - val_loss: 0.2929 - val_accuracy: 0.8939
Epoch 58/100
462/462 [==============================] - 0s 149us/step - loss: 0.2233 - accuracy: 0.9091 - val_loss: 0.2519 - val_accuracy: 0.8687
Epoch 59/100
462/462 [==============================] - 0s 135us/step - loss: 0.2012 - accuracy: 0.9113 - val_loss: 0.2449 - val_accuracy: 0.8838
Epoch 60/100
462/462 [==============================] - 0s 184us/step - loss: 0.2067 - accuracy: 0.9091 - val_loss: 0.2386 - val_accuracy: 0.9040
Epoch 61/100
462/462 [==============================] - 0s 671us/step - loss: 0.2325 - accuracy: 0.9113 - val_loss: 0.2864 - val_accuracy: 0.8232
Epoch 62/100
462/462 [==============================] - 0s 424us/step - loss: 0.2731 - accuracy: 0.8593 - val_loss: 0.3631 - val_accuracy: 0.8788
Epoch 63/100
462/462 [==============================] - 0s 285us/step - loss: 0.2515 - accuracy: 0.8918 - val_loss: 0.2680 -

In [287]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over2.history['accuracy'])*100))

over-sampling train accuracy: 90.70%


In [36]:
df_proba2 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=1,
                        index_col=None)

In [37]:
df_proba2

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,NRS148,2,2,0.000056,1.748042e-03,9.981960e-01
1,p002ykpresabs_qual,BCH-SA-03,1,0,0.712007,2.879924e-01,9.646217e-07
2,p002ykpresabs_qual,NRS218,1,1,0.006222,9.937732e-01,4.482882e-06
3,p002ykpresabs_qual,NRS036,0,0,0.882617,1.173831e-01,2.310933e-10
4,p002ykpresabs_qual,NRS386,1,0,0.571179,4.288184e-01,2.444667e-06
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS112,1,1,0.001860,9.979747e-01,1.653396e-04
4280,pyopresabsSTCC_qual,SR1065,0,0,0.982940,1.705227e-02,7.349168e-06
4281,pyopresabsSTCC_qual,NRS203,0,0,0.997093,1.962516e-03,9.441347e-04
4282,pyopresabsSTCC_qual,CFBREBSa129,0,0,1.000000,3.031141e-13,3.208205e-09


In [38]:
y_prob2 = df_proba2[df_proba2['phage']=='p002ykpresabs_qual'].iloc[:,-3:]
y_prob2 = y_prob2.to_numpy()
y_prob2

array([[5.6032226e-05, 1.7480415e-03, 9.9819595e-01],
       [7.1200660e-01, 2.8799242e-01, 9.6462170e-07],
       [6.2223645e-03, 9.9377316e-01, 4.4828816e-06],
       [8.8261690e-01, 1.1738314e-01, 2.3109333e-10],
       [5.7117930e-01, 4.2881835e-01, 2.4446672e-06],
       [2.6919745e-02, 9.7307040e-01, 9.9069590e-06],
       [2.3617710e-02, 9.7637650e-01, 5.7737575e-06],
       [4.7585876e-03, 3.3258020e-02, 9.6198344e-01],
       [1.2852755e-02, 9.8714060e-01, 6.6576117e-06],
       [4.7585876e-03, 3.3258020e-02, 9.6198344e-01],
       [5.6032226e-05, 1.7480415e-03, 9.9819595e-01],
       [6.5726460e-01, 3.4273115e-01, 4.3337404e-06],
       [2.0976942e-02, 9.7900280e-01, 2.0353025e-05],
       [7.1200660e-01, 2.8799242e-01, 9.6462170e-07],
       [1.2631357e-01, 8.7362730e-01, 5.9109418e-05],
       [7.1200660e-01, 2.8799242e-01, 9.6462170e-07],
       [5.6032226e-05, 1.7480415e-03, 9.9819595e-01],
       [2.5615154e-07, 9.1859310e-05, 9.9990785e-01],
       [1.1462224e-01, 8.853

In [39]:
ovo2 = rocauc_ovo(y_test_over, y_prob2, average="macro", multi_class="ovo")
ovo2

0.9618916437098255

In [40]:
ovr2 = rocauc_ovr(y_test_over, y_prob2, average="macro", multi_class="ovr")
ovr2

0.9618916437098255

In [41]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=234,
                                                    stratify=y_over)

In [42]:
dat3 = pd.DataFrame(X_test_over[:,0])
dat3['test'] = y_test_over

In [43]:
dat3

,0,test
0,NRS109,2
1,NRS109,2
2,NRS222,0
3,NRS109,2
4,GA50245,0
...,...,...
193,NRS148,2
194,NRS266,1
195,NRS109,2
196,NRS149,0


In [44]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [46]:
model1_over3 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [47]:
model1_over3.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [48]:
model1_over3.fit(X_train_over, y_train_over,
          batch_size=64, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 462 samples, validate on 198 samples
Epoch 1/100
462/462 [==============================] - 0s 336us/step - loss: 0.8527 - accuracy: 0.5758 - val_loss: 0.6423 - val_accuracy: 0.6869
Epoch 2/100
462/462 [==============================] - 0s 135us/step - loss: 0.5962 - accuracy: 0.7359 - val_loss: 0.4962 - val_accuracy: 0.7828
Epoch 3/100
462/462 [==============================] - 0s 147us/step - loss: 0.5193 - accuracy: 0.7965 - val_loss: 0.5243 - val_accuracy: 0.7121
Epoch 4/100
462/462 [==============================] - 0s 155us/step - loss: 0.5239 - accuracy: 0.7489 - val_loss: 0.4577 - val_accuracy: 0.8131
Epoch 5/100
462/462 [==============================] - 0s 188us/step - loss: 0.4593 - accuracy: 0.7792 - val_loss: 0.4620 - val_accuracy: 0.8131
Epoch 6/100
462/462 [==============================] - 0s 189us/step - loss: 0.4377 - accuracy: 0.8203 - val_loss: 0.4450 - val_accuracy: 0.8081
Epoch 7/100
462/462 [==============================] - 0s 225us/step - loss: 0.4090 

Epoch 57/100
462/462 [==============================] - 0s 119us/step - loss: 0.2571 - accuracy: 0.9026 - val_loss: 0.2934 - val_accuracy: 0.8788
Epoch 58/100
462/462 [==============================] - 0s 118us/step - loss: 0.2520 - accuracy: 0.8918 - val_loss: 0.2697 - val_accuracy: 0.8889
Epoch 59/100
462/462 [==============================] - 0s 121us/step - loss: 0.2484 - accuracy: 0.9026 - val_loss: 0.2809 - val_accuracy: 0.8939
Epoch 60/100
462/462 [==============================] - 0s 120us/step - loss: 0.2538 - accuracy: 0.9069 - val_loss: 0.2950 - val_accuracy: 0.8788
Epoch 61/100
462/462 [==============================] - 0s 121us/step - loss: 0.2462 - accuracy: 0.8745 - val_loss: 0.2889 - val_accuracy: 0.9040
Epoch 62/100
462/462 [==============================] - 0s 123us/step - loss: 0.2648 - accuracy: 0.8810 - val_loss: 0.2873 - val_accuracy: 0.8939
Epoch 63/100
462/462 [==============================] - 0s 128us/step - loss: 0.2437 - accuracy: 0.9026 - val_loss: 0.2789 -

In [307]:
acc_test_over3 = model1_over3.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over3*100))

198/198 [==============================] - 0s 166us/step
over-sampling test accuracy: 88.38%


In [49]:
pred3 = model1_over3.predict_classes(X_test_over)
pred3

array([2, 2, 0, 2, 0, 2, 0, 1, 0, 2, 0, 1, 0, 1, 0, 2, 1, 2, 1, 2, 0, 1,
       2, 1, 1, 2, 1, 2, 0, 0, 2, 0, 1, 1, 1, 1, 0, 1, 2, 0, 0, 0, 2, 0,
       2, 0, 0, 2, 1, 0, 1, 2, 0, 1, 0, 2, 2, 0, 2, 0, 0, 1, 1, 2, 0, 2,
       0, 0, 0, 0, 1, 1, 2, 1, 2, 2, 0, 0, 0, 1, 1, 1, 0, 0, 2, 0, 0, 0,
       2, 0, 2, 1, 1, 0, 2, 2, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 2, 0, 2, 1,
       0, 2, 1, 1, 1, 1, 0, 0, 0, 2, 0, 2, 1, 2, 1, 2, 2, 2, 1, 0, 1, 1,
       0, 1, 2, 2, 0, 2, 0, 2, 2, 0, 1, 2, 0, 0, 1, 2, 2, 2, 0, 2, 1, 0,
       0, 2, 2, 2, 1, 1, 0, 2, 1, 2, 0, 1, 2, 0, 2, 2, 1, 1, 1, 2, 1, 2,
       0, 0, 0, 1, 2, 1, 2, 0, 0, 0, 0, 0, 1, 2, 2, 1, 1, 2, 1, 2, 0, 2])

In [50]:
dat3['pred'] = pred3
dat3

,0,test,pred
0,NRS109,2,2
1,NRS109,2,2
2,NRS222,0,0
3,NRS109,2,2
4,GA50245,0,0
...,...,...,...
193,NRS148,2,2
194,NRS266,1,1
195,NRS109,2,2
196,NRS149,0,0


In [51]:
proba3 = model1_over3.predict_proba(X_test_over)
dat_proba3 = pd.DataFrame(proba3)

In [52]:
dat_proba3

,0,1,2
0,0.004477,0.013518,9.820048e-01
1,0.004477,0.013518,9.820048e-01
2,0.851725,0.148269,5.980786e-06
3,0.004477,0.013518,9.820048e-01
4,0.812055,0.187945,1.161034e-07
...,...,...,...
193,0.000300,0.000557,9.991430e-01
194,0.041512,0.958437,5.077456e-05
195,0.004477,0.013518,9.820048e-01
196,0.871377,0.128623,1.330259e-07


In [53]:
dat_proba3.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba3.csv", index = False,
         header=None)

In [54]:
dat3.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/3p002.csv", index = False,
         header=None)

In [311]:
hist1_over3 = model1_over3.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 462 samples, validate on 198 samples
Epoch 1/100
462/462 [==============================] - 0s 213us/step - loss: 0.2513 - accuracy: 0.8983 - val_loss: 0.2574 - val_accuracy: 0.8889
Epoch 2/100
462/462 [==============================] - 0s 205us/step - loss: 0.2575 - accuracy: 0.8788 - val_loss: 0.3370 - val_accuracy: 0.8485
Epoch 3/100
462/462 [==============================] - 0s 174us/step - loss: 0.2691 - accuracy: 0.8723 - val_loss: 0.3495 - val_accuracy: 0.8131
Epoch 4/100
462/462 [==============================] - 0s 171us/step - loss: 0.2392 - accuracy: 0.8918 - val_loss: 0.2497 - val_accuracy: 0.8939
Epoch 5/100
462/462 [==============================] - 0s 157us/step - loss: 0.2518 - accuracy: 0.8788 - val_loss: 0.2587 - val_accuracy: 0.8838
Epoch 6/100
462/462 [==============================] - 0s 155us/step - loss: 0.2362 - accuracy: 0.8874 - val_loss: 0.2614 - val_accuracy: 0.8990
Epoch 7/100
462/462 [==============================] - 0s 178us/step - loss: 0.2241 

Epoch 57/100
462/462 [==============================] - 0s 201us/step - loss: 0.2106 - accuracy: 0.9026 - val_loss: 0.2341 - val_accuracy: 0.8939
Epoch 58/100
462/462 [==============================] - 0s 600us/step - loss: 0.2271 - accuracy: 0.9091 - val_loss: 0.2428 - val_accuracy: 0.8889
Epoch 59/100
462/462 [==============================] - 0s 322us/step - loss: 0.2109 - accuracy: 0.9134 - val_loss: 0.3143 - val_accuracy: 0.8535
Epoch 60/100
462/462 [==============================] - 0s 182us/step - loss: 0.2187 - accuracy: 0.9113 - val_loss: 0.3052 - val_accuracy: 0.8131
Epoch 61/100
462/462 [==============================] - 0s 201us/step - loss: 0.2229 - accuracy: 0.8831 - val_loss: 0.2488 - val_accuracy: 0.8889
Epoch 62/100
462/462 [==============================] - 0s 189us/step - loss: 0.2079 - accuracy: 0.9091 - val_loss: 0.2730 - val_accuracy: 0.8838
Epoch 63/100
462/462 [==============================] - 0s 187us/step - loss: 0.2166 - accuracy: 0.9026 - val_loss: 0.2681 -

In [312]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over3.history['accuracy'])*100))

over-sampling train accuracy: 90.16%


In [45]:
df_proba3 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=2,
                        index_col=None)

In [46]:
df_proba3

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,NRS109,2,2,0.004477,0.013518,9.820048e-01
1,p002ykpresabs_qual,NRS109,2,2,0.004477,0.013518,9.820048e-01
2,p002ykpresabs_qual,NRS222,0,0,0.851725,0.148269,5.980786e-06
3,p002ykpresabs_qual,NRS109,2,2,0.004477,0.013518,9.820048e-01
4,p002ykpresabs_qual,GA50245,0,0,0.812055,0.187945,1.161034e-07
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS255,2,2,0.000633,0.000928,9.984396e-01
4280,pyopresabsSTCC_qual,NRS255,2,2,0.000633,0.000928,9.984396e-01
4281,pyopresabsSTCC_qual,NRS266,1,1,0.025932,0.974061,7.323514e-06
4282,pyopresabsSTCC_qual,NRS001,1,1,0.000597,0.999403,3.675362e-10


In [47]:
y_prob3 = df_proba3[df_proba3['phage']=='p002ykpresabs_qual'].iloc[:,-3:]
y_prob3 = y_prob3.to_numpy()
y_prob3

array([[4.47713400e-03, 1.35180650e-02, 9.82004760e-01],
       [4.47713400e-03, 1.35180650e-02, 9.82004760e-01],
       [8.51724570e-01, 1.48269460e-01, 5.98078580e-06],
       [4.47713400e-03, 1.35180650e-02, 9.82004760e-01],
       [8.12054800e-01, 1.87945080e-01, 1.16103394e-07],
       [4.47713400e-03, 1.35180650e-02, 9.82004760e-01],
       [8.12054800e-01, 1.87945080e-01, 1.16103394e-07],
       [3.97700700e-02, 9.60229460e-01, 4.67898960e-07],
       [9.91591330e-01, 8.40804100e-03, 6.02038140e-07],
       [4.47713400e-03, 1.35180650e-02, 9.82004760e-01],
       [9.99479230e-01, 5.20777300e-04, 9.20412000e-09],
       [1.39539150e-02, 9.86045800e-01, 3.59996450e-07],
       [8.12054800e-01, 1.87945080e-01, 1.16103394e-07],
       [5.25667500e-03, 9.71091600e-01, 2.36517100e-02],
       [8.12054800e-01, 1.87945080e-01, 1.16103394e-07],
       [3.00113980e-04, 5.56867800e-04, 9.99143000e-01],
       [3.01311900e-02, 9.69867700e-01, 1.02931100e-06],
       [1.06684830e-07, 1.33550

In [49]:
ovo3 = rocauc_ovo(y_test_over, y_prob3, average="macro", multi_class="ovo")
ovo3

0.9738292011019284

In [50]:
ovr3 = rocauc_ovr(y_test_over, y_prob3, average="macro", multi_class="ovr")
ovr3

0.9738292011019284

In [51]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=456,
                                                    stratify=y_over)

In [52]:
dat4 = pd.DataFrame(X_test_over[:,0])
dat4['test'] = y_test_over

In [53]:
dat4

,0,test
0,NRS110,1
1,NRS216,1
2,NRS386,1
3,CFBRSa25,0
4,BCH-SA-03,1
...,...,...
193,NRS216,1
194,CFBREBSa110,0
195,NRS148,2
196,GA27,0


In [54]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [59]:
model1_over4 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [60]:
model1_over4.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [61]:
model1_over4.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 462 samples, validate on 198 samples
Epoch 1/1000
462/462 [==============================] - 0s 415us/step - loss: 0.7999 - accuracy: 0.5996 - val_loss: 0.5715 - val_accuracy: 0.7071
Epoch 2/1000
462/462 [==============================] - 0s 177us/step - loss: 0.5624 - accuracy: 0.7251 - val_loss: 0.5103 - val_accuracy: 0.7323
Epoch 3/1000
462/462 [==============================] - 0s 198us/step - loss: 0.5086 - accuracy: 0.7489 - val_loss: 0.4776 - val_accuracy: 0.7879
Epoch 4/1000
462/462 [==============================] - 0s 190us/step - loss: 0.4656 - accuracy: 0.7922 - val_loss: 0.4894 - val_accuracy: 0.7727
Epoch 5/1000
462/462 [==============================] - 0s 190us/step - loss: 0.4558 - accuracy: 0.7987 - val_loss: 0.5317 - val_accuracy: 0.7020
Epoch 6/1000
462/462 [==============================] - 0s 219us/step - loss: 0.4558 - accuracy: 0.8052 - val_loss: 0.5531 - val_accuracy: 0.7071
Epoch 7/1000
462/462 [==============================] - 0s 272us/step - loss: 

Epoch 112/1000
462/462 [==============================] - 0s 152us/step - loss: 0.1848 - accuracy: 0.9156 - val_loss: 0.3045 - val_accuracy: 0.8586
Epoch 113/1000
462/462 [==============================] - 0s 174us/step - loss: 0.1857 - accuracy: 0.9134 - val_loss: 0.2949 - val_accuracy: 0.8687
Epoch 114/1000
462/462 [==============================] - 0s 188us/step - loss: 0.1848 - accuracy: 0.9286 - val_loss: 0.3024 - val_accuracy: 0.8737
Epoch 115/1000
462/462 [==============================] - 0s 153us/step - loss: 0.1790 - accuracy: 0.9242 - val_loss: 0.2979 - val_accuracy: 0.8687
Epoch 116/1000
462/462 [==============================] - 0s 151us/step - loss: 0.1903 - accuracy: 0.9242 - val_loss: 0.3013 - val_accuracy: 0.8535
Epoch 117/1000
462/462 [==============================] - 0s 150us/step - loss: 0.1880 - accuracy: 0.9177 - val_loss: 0.3087 - val_accuracy: 0.8737
Epoch 118/1000
462/462 [==============================] - 0s 149us/step - loss: 0.1811 - accuracy: 0.9264 - val_

462/462 [==============================] - 0s 198us/step - loss: 0.1823 - accuracy: 0.9242 - val_loss: 0.3099 - val_accuracy: 0.8535
Epoch 223/1000
462/462 [==============================] - 0s 210us/step - loss: 0.1640 - accuracy: 0.9286 - val_loss: 0.2995 - val_accuracy: 0.8737
Epoch 224/1000
462/462 [==============================] - 0s 180us/step - loss: 0.1957 - accuracy: 0.9026 - val_loss: 0.2920 - val_accuracy: 0.8737
Epoch 225/1000
462/462 [==============================] - 0s 176us/step - loss: 0.1860 - accuracy: 0.9221 - val_loss: 0.3292 - val_accuracy: 0.8434
Epoch 226/1000
462/462 [==============================] - 0s 186us/step - loss: 0.1766 - accuracy: 0.9286 - val_loss: 0.3421 - val_accuracy: 0.7879
Epoch 227/1000
462/462 [==============================] - 0s 192us/step - loss: 0.1748 - accuracy: 0.9091 - val_loss: 0.2913 - val_accuracy: 0.8838
Epoch 228/1000
462/462 [==============================] - 0s 205us/step - loss: 0.1629 - accuracy: 0.9264 - val_loss: 0.2973 - 

462/462 [==============================] - 0s 171us/step - loss: 0.1549 - accuracy: 0.9264 - val_loss: 0.2955 - val_accuracy: 0.8687
Epoch 333/1000
462/462 [==============================] - 0s 180us/step - loss: 0.1655 - accuracy: 0.9221 - val_loss: 0.2890 - val_accuracy: 0.8788
Epoch 334/1000
462/462 [==============================] - 0s 205us/step - loss: 0.1634 - accuracy: 0.9264 - val_loss: 0.3205 - val_accuracy: 0.8535
Epoch 335/1000
462/462 [==============================] - 0s 180us/step - loss: 0.1560 - accuracy: 0.9307 - val_loss: 0.2985 - val_accuracy: 0.8687
Epoch 336/1000
462/462 [==============================] - 0s 195us/step - loss: 0.1803 - accuracy: 0.8983 - val_loss: 0.3476 - val_accuracy: 0.8687
Epoch 337/1000
462/462 [==============================] - 0s 270us/step - loss: 0.1599 - accuracy: 0.9177 - val_loss: 0.3394 - val_accuracy: 0.7828
Epoch 338/1000
462/462 [==============================] - 0s 190us/step - loss: 0.1689 - accuracy: 0.9113 - val_loss: 0.3139 - 

462/462 [==============================] - 0s 504us/step - loss: 0.1622 - accuracy: 0.9242 - val_loss: 0.2997 - val_accuracy: 0.8737
Epoch 443/1000
462/462 [==============================] - 0s 684us/step - loss: 0.1560 - accuracy: 0.9286 - val_loss: 0.2933 - val_accuracy: 0.8838
Epoch 444/1000
462/462 [==============================] - 0s 675us/step - loss: 0.1556 - accuracy: 0.9329 - val_loss: 0.3278 - val_accuracy: 0.8434
Epoch 445/1000
462/462 [==============================] - 0s 579us/step - loss: 0.1581 - accuracy: 0.9286 - val_loss: 0.2892 - val_accuracy: 0.8737
Epoch 446/1000
462/462 [==============================] - 0s 292us/step - loss: 0.1657 - accuracy: 0.9307 - val_loss: 0.2950 - val_accuracy: 0.8737
Epoch 447/1000
462/462 [==============================] - 0s 290us/step - loss: 0.1627 - accuracy: 0.9372 - val_loss: 0.3092 - val_accuracy: 0.8535
Epoch 448/1000
462/462 [==============================] - 0s 281us/step - loss: 0.1612 - accuracy: 0.9264 - val_loss: 0.2931 - 

462/462 [==============================] - 0s 208us/step - loss: 0.1591 - accuracy: 0.9113 - val_loss: 0.2895 - val_accuracy: 0.8737
Epoch 553/1000
462/462 [==============================] - 0s 235us/step - loss: 0.1645 - accuracy: 0.9329 - val_loss: 0.3015 - val_accuracy: 0.8788
Epoch 554/1000
462/462 [==============================] - 0s 215us/step - loss: 0.1577 - accuracy: 0.9329 - val_loss: 0.2914 - val_accuracy: 0.8788
Epoch 555/1000
462/462 [==============================] - 0s 212us/step - loss: 0.1577 - accuracy: 0.9264 - val_loss: 0.3119 - val_accuracy: 0.8586
Epoch 556/1000
462/462 [==============================] - 0s 272us/step - loss: 0.1605 - accuracy: 0.9264 - val_loss: 0.3004 - val_accuracy: 0.8838
Epoch 557/1000
462/462 [==============================] - 0s 316us/step - loss: 0.1567 - accuracy: 0.9286 - val_loss: 0.2919 - val_accuracy: 0.8788
Epoch 558/1000
462/462 [==============================] - 0s 234us/step - loss: 0.1526 - accuracy: 0.9351 - val_loss: 0.3119 - 

462/462 [==============================] - 0s 280us/step - loss: 0.1498 - accuracy: 0.9286 - val_loss: 0.3088 - val_accuracy: 0.8737
Epoch 663/1000
462/462 [==============================] - 0s 261us/step - loss: 0.1727 - accuracy: 0.9199 - val_loss: 0.3528 - val_accuracy: 0.8333
Epoch 664/1000
462/462 [==============================] - 0s 252us/step - loss: 0.1673 - accuracy: 0.9329 - val_loss: 0.3093 - val_accuracy: 0.8838
Epoch 665/1000
462/462 [==============================] - 0s 313us/step - loss: 0.1558 - accuracy: 0.9286 - val_loss: 0.3033 - val_accuracy: 0.8788
Epoch 666/1000
462/462 [==============================] - 0s 209us/step - loss: 0.1575 - accuracy: 0.9351 - val_loss: 0.3342 - val_accuracy: 0.8434
Epoch 667/1000
462/462 [==============================] - 0s 271us/step - loss: 0.1599 - accuracy: 0.9329 - val_loss: 0.2994 - val_accuracy: 0.8687
Epoch 668/1000
462/462 [==============================] - 0s 206us/step - loss: 0.1614 - accuracy: 0.9264 - val_loss: 0.3083 - 

Epoch 717/1000
462/462 [==============================] - 0s 234us/step - loss: 0.1620 - accuracy: 0.9242 - val_loss: 0.2977 - val_accuracy: 0.8737
Epoch 718/1000
462/462 [==============================] - 0s 224us/step - loss: 0.1629 - accuracy: 0.9264 - val_loss: 0.3091 - val_accuracy: 0.8788
Epoch 719/1000
462/462 [==============================] - 0s 307us/step - loss: 0.1505 - accuracy: 0.9416 - val_loss: 0.3565 - val_accuracy: 0.8333
Epoch 720/1000
462/462 [==============================] - 0s 248us/step - loss: 0.1667 - accuracy: 0.9286 - val_loss: 0.3090 - val_accuracy: 0.8586
Epoch 721/1000
462/462 [==============================] - 0s 252us/step - loss: 0.1510 - accuracy: 0.9264 - val_loss: 0.2991 - val_accuracy: 0.8636
Epoch 722/1000
462/462 [==============================] - 0s 245us/step - loss: 0.1484 - accuracy: 0.9329 - val_loss: 0.3029 - val_accuracy: 0.8636
Epoch 723/1000
462/462 [==============================] - 0s 217us/step - loss: 0.1495 - accuracy: 0.9307 - val_

462/462 [==============================] - 0s 527us/step - loss: 0.1497 - accuracy: 0.9242 - val_loss: 0.3212 - val_accuracy: 0.8687
Epoch 828/1000
462/462 [==============================] - 0s 449us/step - loss: 0.1593 - accuracy: 0.9307 - val_loss: 0.3577 - val_accuracy: 0.8485
Epoch 829/1000
462/462 [==============================] - 0s 454us/step - loss: 0.1481 - accuracy: 0.9199 - val_loss: 0.3179 - val_accuracy: 0.8586
Epoch 830/1000
462/462 [==============================] - 0s 329us/step - loss: 0.1591 - accuracy: 0.9307 - val_loss: 0.3588 - val_accuracy: 0.8485
Epoch 831/1000
462/462 [==============================] - 0s 295us/step - loss: 0.1579 - accuracy: 0.9264 - val_loss: 0.3236 - val_accuracy: 0.8586
Epoch 832/1000
462/462 [==============================] - 0s 339us/step - loss: 0.1507 - accuracy: 0.9329 - val_loss: 0.3207 - val_accuracy: 0.8687
Epoch 833/1000
462/462 [==============================] - 0s 412us/step - loss: 0.1529 - accuracy: 0.9307 - val_loss: 0.3341 - 

462/462 [==============================] - 0s 481us/step - loss: 0.1556 - accuracy: 0.9307 - val_loss: 0.3945 - val_accuracy: 0.8485
Epoch 938/1000
462/462 [==============================] - 0s 407us/step - loss: 0.1525 - accuracy: 0.9242 - val_loss: 0.3614 - val_accuracy: 0.8737
Epoch 939/1000
462/462 [==============================] - 0s 301us/step - loss: 0.1570 - accuracy: 0.9351 - val_loss: 0.4278 - val_accuracy: 0.8434
Epoch 940/1000
462/462 [==============================] - 0s 297us/step - loss: 0.1498 - accuracy: 0.9264 - val_loss: 0.3597 - val_accuracy: 0.8687
Epoch 941/1000
462/462 [==============================] - 0s 196us/step - loss: 0.1474 - accuracy: 0.9221 - val_loss: 0.3649 - val_accuracy: 0.8586
Epoch 942/1000
462/462 [==============================] - 0s 304us/step - loss: 0.1460 - accuracy: 0.9329 - val_loss: 0.3669 - val_accuracy: 0.8586
Epoch 943/1000
462/462 [==============================] - 0s 276us/step - loss: 0.1460 - accuracy: 0.9307 - val_loss: 0.3628 - 

In [374]:
acc_test_over4 = model1_over4.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over4*100))

198/198 [==============================] - 0s 181us/step
over-sampling test accuracy: 87.37%


In [64]:
pred4 = model1_over4.predict_classes(X_test_over)
pred4

array([1, 1, 1, 0, 0, 1, 2, 2, 2, 1, 2, 0, 0, 0, 0, 2, 0, 2, 1, 0, 2, 1,
       0, 2, 1, 1, 2, 1, 2, 1, 1, 2, 1, 2, 0, 1, 0, 0, 2, 0, 0, 1, 1, 1,
       0, 1, 0, 2, 2, 0, 0, 2, 1, 2, 2, 1, 1, 0, 0, 0, 1, 0, 2, 0, 2, 2,
       0, 0, 0, 2, 2, 0, 0, 1, 2, 0, 0, 2, 1, 0, 1, 1, 2, 1, 2, 0, 0, 1,
       0, 0, 2, 2, 2, 2, 2, 1, 1, 1, 1, 0, 2, 0, 0, 1, 2, 2, 1, 0, 2, 0,
       0, 2, 2, 2, 0, 2, 0, 2, 0, 2, 0, 0, 2, 0, 1, 0, 1, 2, 2, 2, 0, 1,
       0, 2, 1, 2, 0, 0, 0, 2, 1, 2, 2, 1, 1, 2, 0, 1, 1, 0, 0, 0, 1, 2,
       2, 2, 1, 1, 0, 2, 2, 0, 0, 2, 1, 0, 1, 2, 0, 1, 1, 0, 1, 1, 0, 0,
       0, 0, 0, 2, 1, 0, 1, 0, 2, 1, 0, 0, 0, 2, 0, 2, 2, 1, 0, 2, 0, 2])

In [65]:
dat4['pred'] = pred4
dat4

,0,test,pred
0,NRS110,1,1
1,NRS216,1,1
2,NRS386,1,1
3,CFBRSa25,0,0
4,BCH-SA-03,1,0
...,...,...,...
193,NRS216,1,1
194,CFBREBSa110,0,0
195,NRS148,2,2
196,GA27,0,0


In [66]:
proba4 = model1_over4.predict_proba(X_test_over)
dat_proba4 = pd.DataFrame(proba4)

In [67]:
dat_proba4

,0,1,2
0,2.716965e-06,9.999973e-01,5.870196e-13
1,3.925433e-02,9.607447e-01,9.078969e-07
2,3.267520e-01,6.732479e-01,1.061032e-07
3,6.110835e-01,3.889157e-01,7.664974e-07
4,6.110835e-01,3.889157e-01,7.664974e-07
...,...,...,...
193,3.925431e-02,9.607448e-01,9.078979e-07
194,6.110834e-01,3.889157e-01,7.664980e-07
195,2.844958e-13,4.825762e-08,1.000000e+00
196,9.531750e-01,4.682501e-02,8.638050e-27


In [68]:
dat_proba4.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba4.csv", index = False,
         header=None)

In [69]:
dat4.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/4p002.csv", index = False,
         header=None)

In [378]:
hist1_over4 = model1_over4.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 462 samples, validate on 198 samples
Epoch 1/1000
462/462 [==============================] - 0s 266us/step - loss: 0.1542 - accuracy: 0.9307 - val_loss: 0.3291 - val_accuracy: 0.8687
Epoch 2/1000
462/462 [==============================] - 0s 190us/step - loss: 0.1444 - accuracy: 0.9329 - val_loss: 0.3713 - val_accuracy: 0.8636
Epoch 3/1000
462/462 [==============================] - 0s 211us/step - loss: 0.1475 - accuracy: 0.9307 - val_loss: 0.3418 - val_accuracy: 0.8687
Epoch 4/1000
462/462 [==============================] - 0s 522us/step - loss: 0.1482 - accuracy: 0.9307 - val_loss: 0.3513 - val_accuracy: 0.8687
Epoch 5/1000
462/462 [==============================] - 0s 324us/step - loss: 0.1445 - accuracy: 0.9307 - val_loss: 0.3437 - val_accuracy: 0.8737
Epoch 6/1000
462/462 [==============================] - 0s 205us/step - loss: 0.1565 - accuracy: 0.9329 - val_loss: 0.3579 - val_accuracy: 0.8636
Epoch 7/1000
462/462 [==============================] - 0s 187us/step - loss: 

Epoch 112/1000
462/462 [==============================] - 0s 224us/step - loss: 0.1439 - accuracy: 0.9307 - val_loss: 0.3429 - val_accuracy: 0.8737
Epoch 113/1000
462/462 [==============================] - 0s 214us/step - loss: 0.1505 - accuracy: 0.9372 - val_loss: 0.3474 - val_accuracy: 0.8636
Epoch 114/1000
462/462 [==============================] - 0s 209us/step - loss: 0.1449 - accuracy: 0.9329 - val_loss: 0.3355 - val_accuracy: 0.8737
Epoch 115/1000
462/462 [==============================] - 0s 189us/step - loss: 0.1503 - accuracy: 0.9307 - val_loss: 0.3269 - val_accuracy: 0.8687
Epoch 116/1000
462/462 [==============================] - 0s 177us/step - loss: 0.1506 - accuracy: 0.9329 - val_loss: 0.3446 - val_accuracy: 0.8636
Epoch 117/1000
462/462 [==============================] - 0s 168us/step - loss: 0.1451 - accuracy: 0.9264 - val_loss: 0.3324 - val_accuracy: 0.8737
Epoch 118/1000
462/462 [==============================] - 0s 194us/step - loss: 0.1464 - accuracy: 0.9307 - val_

462/462 [==============================] - 0s 249us/step - loss: 0.1476 - accuracy: 0.9351 - val_loss: 0.3753 - val_accuracy: 0.8687
Epoch 223/1000
462/462 [==============================] - 0s 220us/step - loss: 0.1443 - accuracy: 0.9307 - val_loss: 0.3659 - val_accuracy: 0.8737
Epoch 224/1000
462/462 [==============================] - 0s 319us/step - loss: 0.1550 - accuracy: 0.9329 - val_loss: 0.3902 - val_accuracy: 0.8737
Epoch 225/1000
462/462 [==============================] - 0s 256us/step - loss: 0.1453 - accuracy: 0.9351 - val_loss: 0.3562 - val_accuracy: 0.8737
Epoch 226/1000
462/462 [==============================] - 0s 227us/step - loss: 0.1471 - accuracy: 0.9307 - val_loss: 0.3683 - val_accuracy: 0.8737
Epoch 227/1000
462/462 [==============================] - 0s 228us/step - loss: 0.1453 - accuracy: 0.9329 - val_loss: 0.3721 - val_accuracy: 0.8737
Epoch 228/1000
462/462 [==============================] - 0s 256us/step - loss: 0.1453 - accuracy: 0.9329 - val_loss: 0.3617 - 

462/462 [==============================] - 0s 555us/step - loss: 0.1441 - accuracy: 0.9307 - val_loss: 0.3400 - val_accuracy: 0.8687
Epoch 333/1000
462/462 [==============================] - 0s 290us/step - loss: 0.1512 - accuracy: 0.9307 - val_loss: 0.3549 - val_accuracy: 0.8636
Epoch 334/1000
462/462 [==============================] - 0s 265us/step - loss: 0.1473 - accuracy: 0.9307 - val_loss: 0.3618 - val_accuracy: 0.8636
Epoch 335/1000
462/462 [==============================] - 0s 224us/step - loss: 0.1452 - accuracy: 0.9329 - val_loss: 0.3451 - val_accuracy: 0.8737
Epoch 336/1000
462/462 [==============================] - 0s 195us/step - loss: 0.1460 - accuracy: 0.9351 - val_loss: 0.3300 - val_accuracy: 0.8687
Epoch 337/1000
462/462 [==============================] - 0s 205us/step - loss: 0.1465 - accuracy: 0.9307 - val_loss: 0.3599 - val_accuracy: 0.8636
Epoch 338/1000
462/462 [==============================] - 0s 194us/step - loss: 0.1429 - accuracy: 0.9329 - val_loss: 0.3331 - 

462/462 [==============================] - 0s 576us/step - loss: 0.1413 - accuracy: 0.9307 - val_loss: 0.3448 - val_accuracy: 0.8687
Epoch 443/1000
462/462 [==============================] - 0s 249us/step - loss: 0.1441 - accuracy: 0.9329 - val_loss: 0.3404 - val_accuracy: 0.8737
Epoch 444/1000
462/462 [==============================] - 0s 212us/step - loss: 0.1439 - accuracy: 0.9307 - val_loss: 0.3381 - val_accuracy: 0.8737
Epoch 445/1000
462/462 [==============================] - 0s 191us/step - loss: 0.1479 - accuracy: 0.9307 - val_loss: 0.3436 - val_accuracy: 0.8687
Epoch 446/1000
462/462 [==============================] - 0s 285us/step - loss: 0.1413 - accuracy: 0.9307 - val_loss: 0.3411 - val_accuracy: 0.8737
Epoch 447/1000
462/462 [==============================] - 0s 211us/step - loss: 0.1443 - accuracy: 0.9329 - val_loss: 0.3522 - val_accuracy: 0.8737
Epoch 448/1000
462/462 [==============================] - 0s 201us/step - loss: 0.1453 - accuracy: 0.9372 - val_loss: 0.3512 - 

Epoch 552/1000
462/462 [==============================] - 0s 484us/step - loss: 0.1441 - accuracy: 0.9351 - val_loss: 0.3654 - val_accuracy: 0.8687
Epoch 553/1000
462/462 [==============================] - 0s 200us/step - loss: 0.1403 - accuracy: 0.9307 - val_loss: 0.3663 - val_accuracy: 0.8737
Epoch 554/1000
462/462 [==============================] - 0s 170us/step - loss: 0.1436 - accuracy: 0.9351 - val_loss: 0.3788 - val_accuracy: 0.8636
Epoch 555/1000
462/462 [==============================] - 0s 164us/step - loss: 0.1425 - accuracy: 0.9307 - val_loss: 0.3610 - val_accuracy: 0.8687
Epoch 556/1000
462/462 [==============================] - 0s 161us/step - loss: 0.1422 - accuracy: 0.9329 - val_loss: 0.3696 - val_accuracy: 0.8687
Epoch 557/1000
462/462 [==============================] - 0s 161us/step - loss: 0.1428 - accuracy: 0.9307 - val_loss: 0.3639 - val_accuracy: 0.8687
Epoch 558/1000
462/462 [==============================] - 0s 155us/step - loss: 0.1421 - accuracy: 0.9286 - val_

462/462 [==============================] - 0s 181us/step - loss: 0.1398 - accuracy: 0.9307 - val_loss: 0.3670 - val_accuracy: 0.8737
Epoch 663/1000
462/462 [==============================] - 0s 178us/step - loss: 0.1405 - accuracy: 0.9329 - val_loss: 0.3820 - val_accuracy: 0.8586
Epoch 664/1000
462/462 [==============================] - 0s 347us/step - loss: 0.1439 - accuracy: 0.9264 - val_loss: 0.3580 - val_accuracy: 0.8687
Epoch 665/1000
462/462 [==============================] - 0s 222us/step - loss: 0.1413 - accuracy: 0.9329 - val_loss: 0.3692 - val_accuracy: 0.8687
Epoch 666/1000
462/462 [==============================] - 0s 175us/step - loss: 0.1445 - accuracy: 0.9351 - val_loss: 0.3740 - val_accuracy: 0.8586
Epoch 667/1000
462/462 [==============================] - 0s 175us/step - loss: 0.1449 - accuracy: 0.9329 - val_loss: 0.3593 - val_accuracy: 0.8737
Epoch 668/1000
462/462 [==============================] - 0s 171us/step - loss: 0.1426 - accuracy: 0.9264 - val_loss: 0.3704 - 

462/462 [==============================] - 0s 242us/step - loss: 0.1397 - accuracy: 0.9307 - val_loss: 0.3708 - val_accuracy: 0.8788
Epoch 773/1000
462/462 [==============================] - 0s 279us/step - loss: 0.1407 - accuracy: 0.9286 - val_loss: 0.3733 - val_accuracy: 0.8737
Epoch 774/1000
462/462 [==============================] - 0s 205us/step - loss: 0.1419 - accuracy: 0.9307 - val_loss: 0.3839 - val_accuracy: 0.8636
Epoch 775/1000
462/462 [==============================] - 0s 183us/step - loss: 0.1425 - accuracy: 0.9264 - val_loss: 0.3684 - val_accuracy: 0.8737
Epoch 776/1000
462/462 [==============================] - 0s 329us/step - loss: 0.1408 - accuracy: 0.9307 - val_loss: 0.3867 - val_accuracy: 0.8687
Epoch 777/1000
462/462 [==============================] - 0s 269us/step - loss: 0.1418 - accuracy: 0.9307 - val_loss: 0.3926 - val_accuracy: 0.8687
Epoch 778/1000
462/462 [==============================] - 0s 179us/step - loss: 0.1413 - accuracy: 0.9242 - val_loss: 0.3553 - 

462/462 [==============================] - 0s 189us/step - loss: 0.1435 - accuracy: 0.9307 - val_loss: 0.3353 - val_accuracy: 0.8838
Epoch 883/1000
462/462 [==============================] - 0s 178us/step - loss: 0.1430 - accuracy: 0.9307 - val_loss: 0.3252 - val_accuracy: 0.8838
Epoch 884/1000
462/462 [==============================] - 0s 172us/step - loss: 0.1384 - accuracy: 0.9307 - val_loss: 0.3313 - val_accuracy: 0.8788
Epoch 885/1000
462/462 [==============================] - 0s 185us/step - loss: 0.1389 - accuracy: 0.9329 - val_loss: 0.3272 - val_accuracy: 0.8788
Epoch 886/1000
462/462 [==============================] - 0s 172us/step - loss: 0.1389 - accuracy: 0.9307 - val_loss: 0.3448 - val_accuracy: 0.8788
Epoch 887/1000
462/462 [==============================] - 0s 172us/step - loss: 0.1467 - accuracy: 0.9329 - val_loss: 0.3349 - val_accuracy: 0.8788
Epoch 888/1000
462/462 [==============================] - 0s 171us/step - loss: 0.1475 - accuracy: 0.9307 - val_loss: 0.3255 - 

462/462 [==============================] - 0s 229us/step - loss: 0.1480 - accuracy: 0.9307 - val_loss: 0.3297 - val_accuracy: 0.8788
Epoch 993/1000
462/462 [==============================] - 0s 232us/step - loss: 0.1493 - accuracy: 0.9351 - val_loss: 0.3320 - val_accuracy: 0.8788
Epoch 994/1000
462/462 [==============================] - 0s 226us/step - loss: 0.1508 - accuracy: 0.9134 - val_loss: 0.3275 - val_accuracy: 0.8788
Epoch 995/1000
462/462 [==============================] - 0s 172us/step - loss: 0.1426 - accuracy: 0.9351 - val_loss: 0.3276 - val_accuracy: 0.8788
Epoch 996/1000
462/462 [==============================] - 0s 165us/step - loss: 0.1415 - accuracy: 0.9329 - val_loss: 0.3301 - val_accuracy: 0.8788
Epoch 997/1000
462/462 [==============================] - 0s 178us/step - loss: 0.1426 - accuracy: 0.9307 - val_loss: 0.3269 - val_accuracy: 0.8838
Epoch 998/1000
462/462 [==============================] - 0s 180us/step - loss: 0.1508 - accuracy: 0.9351 - val_loss: 0.3275 - 

In [380]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over4.history['accuracy'])*100))

over-sampling train accuracy: 93.08%


In [55]:
df_proba4 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=3,
                        index_col=None)

In [56]:
df_proba4

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,NRS110,1,1,0.000003,0.999997,5.870196e-13
1,p002ykpresabs_qual,NRS216,1,1,0.039254,0.960745,9.078969e-07
2,p002ykpresabs_qual,NRS386,1,1,0.326752,0.673248,1.061032e-07
3,p002ykpresabs_qual,CFBRSa25,0,0,0.611084,0.388916,7.664974e-07
4,p002ykpresabs_qual,BCH-SA-03,1,0,0.611084,0.388916,7.664974e-07
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS236,1,1,0.000052,0.999768,1.803156e-04
4280,pyopresabsSTCC_qual,NRS029,0,1,0.322350,0.677496,1.533154e-04
4281,pyopresabsSTCC_qual,NRS148,2,2,0.000006,0.000026,9.999682e-01
4282,pyopresabsSTCC_qual,CFBRSa28,0,0,0.999288,0.000176,5.361527e-04


In [57]:
y_prob4 = df_proba4[df_proba4['phage']=='p002ykpresabs_qual'].iloc[:,-3:]
y_prob4 = y_prob4.to_numpy()
y_prob4

array([[2.7169650e-06, 9.9999726e-01, 5.8701964e-13],
       [3.9254326e-02, 9.6074474e-01, 9.0789695e-07],
       [3.2675198e-01, 6.7324793e-01, 1.0610322e-07],
       [6.1108350e-01, 3.8891566e-01, 7.6649735e-07],
       [6.1108350e-01, 3.8891566e-01, 7.6649735e-07],
       [2.7169650e-06, 9.9999726e-01, 5.8701964e-13],
       [1.5103468e-13, 5.1703296e-08, 1.0000000e+00],
       [1.5103468e-13, 5.1703296e-08, 1.0000000e+00],
       [1.5103468e-13, 5.1703296e-08, 1.0000000e+00],
       [1.2518988e-04, 9.9987483e-01, 6.8840210e-09],
       [2.8449695e-13, 4.8257710e-08, 1.0000000e+00],
       [9.1594750e-01, 8.4052560e-02, 7.7378335e-09],
       [5.5546330e-01, 4.4453666e-01, 7.9212210e-10],
       [5.9862214e-01, 4.0137768e-01, 2.4271156e-07],
       [6.1108350e-01, 3.8891566e-01, 7.6649735e-07],
       [1.5103468e-13, 5.1703296e-08, 1.0000000e+00],
       [5.8443725e-01, 4.1556278e-01, 1.5960390e-08],
       [1.5103468e-13, 5.1703296e-08, 1.0000000e+00],
       [2.7169650e-06, 9.999

In [58]:
ovo4 = rocauc_ovo(y_test_over, y_prob4, average="macro", multi_class="ovo")
ovo4

0.9521349862258953

In [59]:
ovr4 = rocauc_ovr(y_test_over, y_prob4, average="macro", multi_class="ovr")
ovr4

0.9521349862258953

In [61]:
ovos = [ovo1, ovo2, ovo3, ovo4]
np.mean(ovos)

0.9640342822161003

In [62]:
np.std(ovos)

0.008064764758009895

In [63]:
ovrs = [ovr1, ovr2, ovr3, ovr4]
np.mean(ovrs)

0.9640342822161003

In [69]:
np.std(ovrs)

0.008064764758009895

In [381]:
accs = [acc_test_over, acc_test_over2, acc_test_over3, acc_test_over4]

In [382]:
mean = np.mean(accs)
print('over-sampling test accuracy mean: %.2f%%' % (mean*100))

over-sampling test accuracy mean: 88.64%


In [383]:
std = np.std(accs)
print('over-sampling test accuracy standard deviation:', std)

over-sampling test accuracy standard deviation: 0.01359890927346061


In [384]:
accs_train = [np.mean(hist1_over.history['accuracy']), np.mean(hist1_over2.history['accuracy']), np.mean(hist1_over3.history['accuracy']),
             np.mean(hist1_over4.history['accuracy'])]

In [385]:
mean_train = np.mean(accs_train)
print('over-sampling train accuracy mean: %.2f%%' % (mean_train*100))

over-sampling train accuracy mean: 91.14%


In [386]:
std_train = np.std(accs_train)
print('over-sampling train accuracy standard deviation:', std_train)

over-sampling train accuracy standard deviation: 0.011374833
